**SQLite Class and helper methods**

In [1]:
import string
import os
import sqlite3
from string import Template


def create_SQLite_connection(db):
    """ create a database connection to a SQLite database.
    """
    if type(db) != str:
        raise TypeError('db parameter must be str.')
    try:
        conn = sqlite3.connect(db)
    except Error as e:
        print(e)
    return conn


class SQLiteQuery(object):
    def __init__(self, conn, query):
        """SQLite class used to query.
        
        Args:
            conn (:obj:`sqlite3.Connection`): SQLite database connection created using `create_SQLite_connection` method.
            query (:obj:`str`): Query to run using `sqlite3.Connection` object.
        """
        self._conn = conn
        self._query = query
        
    def run_query(self):
        """Runs self._query using self._conn.
        
        Uses:
            self._conn (:obj:`sqlite3.Connection`): SQLite database connection created using `create_SQLite_connection` method.
            self._query (:obj:`str`): Query to run using `sqlite3.Connection` object.
        """
        cursor = self._conn.cursor()
        cursor.execute(self._query)
        print('query ran successfully!')
        cursor.close()
        
    def run_query_return_list_of_dicts(self):
        """Runs self._query using self._conn returns a list_of_dicts.
        
        Uses:
            self._conn (:obj:`sqlite3.Connection`): SQLite database connection created using `create_SQLite_connection` method.
            self._query (:obj:`str`): Query to run using `sqlite3.Connection` object.
            
        Returns:
            list_of_dicts (:obj:`list` of type `dict`): List of dictionaries returned by running self._query
        """
        list_of_dicts = list()
        cursor = self._conn.cursor()
        
        for row in cursor.execute(self._query):
            d = dict(zip(next(zip(*cursor.description)), row))
            list_of_dicts.append(d)
            
        cursor.close()
        return list_of_dicts
    
    @property
    def query(self):
        """query (:obj:`str`): Query to run using `sqlite3.Connection`."""
        return self._query

    @query.setter
    def query(self, query):
        if type(query) != str:
            raise TypeError('query parameter must be str.')
        self._query = query

    @property
    def conn(self):
        """conn (:obj:`sqlite3.Connection`): SQLite database connection created using `create_SQLite_connection` method."""
        return self._conn

    @conn.setter
    def conn(self, conn):
        if type(conn) != sqlite3.Connection:
            raise TypeError('conn parameter must be type sqlite3.Connection.')
        self._conn = conn

**Neo4j Class and helper methods**

In [2]:
def create_string_from_file(file_path):
    """Reads and returns a string from a file path.
    
    Args:
        file_path (:obj:`str`): File path of the file to be read into a string.
    
    Return:
        s (:obj:`str`): String representation of the contents of the input file located at `file_path`.
    """
    with open(file_path, 'r') as file:
        s = file.read().replace('\n', '')
    return s

def populate_cypher_query_template_from_file(cypher_query_template_file_path, print_cypher_query_template=False):
    """Populates and Returns a user provided Cypher query with values read from file specified at `cypher_query_template_file_path`.
    
    Args:
        cypher_query_template_file_path (:obj:`str`): File path of the .cql query with "$"-based substitution characters present.
        print_cypher_query_template (:obj:`bool`): Print string representation of the contents of the input file located at `cypher_query_template_file_path`.
        
    Return:
        cypher_query_template (:obj:`string.Template`): string.Template with "$"-based substitution characters present.
    """
    cypher_query_template = create_string_from_file(cypher_query_template_file_path)
    if print_cypher_query_template:
        print(cypher_query_template)
    return Template(cypher_query_template)

def populate_cypher_query_template_with_values_in_dict(cypher_query_template, dictionary):
    """Populates and Returns a user provided Cypher query with values plucked from a `dictionary`.
    
    Args:
        cypher_query_template (:obj:`string.Template`): string.Template to populate with values.
        dictionary (:obj:`dict`): dict of form { "$"-based substitution: value to be passed to string.Template }.
    
    Return:
        query (:obj:`str`): Input `cypher_query_template` returned with values passed to "$"-based substitutions present.
    """
    if type(cypher_query_template) != string.Template:
        raise TypeError('cypher_query_template parameter must be string.Template.')
    if type(dictionary) != dict:
        raise TypeError('dictionary parameter must be dict.')
    query = cypher_query_template.substitute(dictionary)
    return query

def list_of_dicts_to_list_of_cypher_queries(cypher_query_template, list_of_dicts):
    """Populates and Returns a list of cypher queries from an input `list_of_dicts`.
    
    Args:
        cypher_query_template (:obj:`string._TemplateMetaclass`): string.Template
        list_of_dicts (:obj:`list` of type `dict`): List of dicts with form { "$"-based substitution: value to be passed to string.Template }
    
    Uses:
        `populate_cypher_query_template_with_values_in_dict` method
    
    Return:
        list_of_cypher_queries (:obj:`list` of type `str`): List of strings of `cypher_query_template` returned with values passed to "$"-based substitutions present.
    """
    
    if type(cypher_query_template) != string.Template:
        raise TypeError('cypher_query_template parameter must be string.Template.')
    if type(list_of_dicts) != list:
        raise TypeError('list_of_dicts parameter must be list of dicts.')
    if not all(isinstance(d,dict) for d in list_of_dicts):
        raise TypeError('list_of_dicts parameter must be list of dicts.')
    
    list_of_cypher_queries = list()
    
    for d in list_of_dicts:
        query = populate_cypher_query_template_with_values_in_dict(cypher_query_template, d)
        list_of_cypher_queries.append(query)
        
    return list_of_cypher_queries

def fill_in_cypher_query_template_from_sqlite_query_results(sqlite_db_path, sqlite_query, cypher_query_template, cypher_statement_delimiter="\n"):
    """ Populates and returns a cypher_query_template populated from an input `sqlite_query`.
        1) Establishes a sqlite3.Connection using sqlite database located at `sqlite_db_path`
        2) Executes `sqlite_query` using a sqlite3.Connection => returns list_of_dicts
        3) Populates Cypher query template `cypher_query_template` using list_of_dicts => returns list_of_cypher_queries
        4) Returns list_of_cypher_queries delimited by `cypher_statement_delimiter`
    
    Args:
        sqlite_db_path (:obj:`str`): Path of a sqlite database to run `sqlite_query` against.
        sqlite_query (:obj:`str`): SQL query to run against `sqlite_db_path`
        cypher_query_template (:obj:`string.Template`): string.Template with "$"-based substitution characters present.
        cypher_statement_delimiter (:obj:`str`): Delimiter to use to seperate Cypher statements
        
    Uses:
        `create_SQLite_connection` method
        `SQLiteQuery` class
            `run_query_return_list_of_dicts` method
        `list_of_dicts_to_list_of_cypher_queries` method
        
    Return:
        cypher_query_string (:obj:`str`): String returned with values passed to "$"-based substitutions present.
    """
    with create_SQLite_connection(sqlite_db_path) as conn:
        list_of_dicts = SQLiteQuery(conn,sqlite_query).run_query_return_list_of_dicts()
    list_of_cypher_queries = list_of_dicts_to_list_of_cypher_queries(cypher_query_template, list_of_dicts)
    cypher_query_string = cypher_statement_delimiter.join(list_of_cypher_queries)
    return cypher_query_string

**Example Usage from Static `list_of_dicts`:**

In [3]:
cypher_query_template_path = os.path.join(os.getcwd(),'query.cql')

print('Blank Cypher Query Template String:\n')

cypher_query_template = populate_cypher_query_template_from_file(cypher_query_template_path, True)

list_of_dicts = [
 {'id': 1,
  'name': 'fam training',
  'begin_date': '2020-02-01',
  'end_date': '2020-02-10'},
    
 {'id': 2,
  'name': 'project1',
  'begin_date': '2020-02-01',
  'end_date': '2020-03-01'},
    
 {'id': 3,
  'name': 'project2',
  'begin_date': '2020-02-15',
  'end_date': '2020-04-01'},
    
 {'id': 4,
  'name': 'project3',
  'begin_date': '2020-03-15',
  'end_date': '2020-05-01'}
]

list_of_cypher_queries = list_of_dicts_to_list_of_cypher_queries(cypher_query_template, list_of_dicts)

print('\nFilled in Cypher Query Template String:\n')
print("\n".join(list_of_cypher_queries))

Blank Cypher Query Template String:

MERGE (p:Project {id:$id})

Filled in Cypher Query Template String:

MERGE (p:Project {id:1})
MERGE (p:Project {id:2})
MERGE (p:Project {id:3})
MERGE (p:Project {id:4})


**Example Usage from result fetched from SQLite Database:**

In [4]:
temp_db_path = os.path.join(os.getcwd(),'temp.db')
sqlite_query_path = os.path.join(os.getcwd(),'select_statement_projects.sql')
cypher_query_template_path = os.path.join(os.getcwd(),'query.cql')

print('Blank Cypher Query Template String:\n')

cypher_query_template = populate_cypher_query_template_from_file(cypher_query_template_path, True)
sqlite_query = create_string_from_file(sqlite_query_path)

cypher_query_string = fill_in_cypher_query_template_from_sqlite_query_results(temp_db_path, sqlite_query, cypher_query_template)

print('\nFilled in Cypher Query Template String:\n')
print(cypher_query_string)

Blank Cypher Query Template String:

MERGE (p:Project {id:$id})

Filled in Cypher Query Template String:

MERGE (p:Project {id:1})
MERGE (p:Project {id:2})
MERGE (p:Project {id:3})
MERGE (p:Project {id:4})
